In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\Asus F15\\Text-summarization\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\Asus F15\\Text-summarization'

In [49]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [98]:
from textsummarizer.constants import *
from textsummarizer.utils.common import read_yaml, create_directories

In [114]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
            config = self.config['data_ingestion']

            create_directories([config['root_dir']])

            data_ingestion_config = DataIngestionConfig(
                root_dir=Path(config['root_dir']),
                source_url=config['source_url'],
                local_data_file=Path(config['local_data_file']),
                unzip_dir=Path(config['unzip_dir'])
            )

            return data_ingestion_config

In [87]:
import os
import urllib.request as request
import zipfile
from textsummarizer.logging import logger
from textsummarizer.utils.common import get_size 

In [112]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
            if not os.path.exists(self.config.local_data_file):
                filename,headers= request.urlretrieve(
                    url=self.config.source_url,
                    filename=self.config.local_data_file
                )
                logger.info(f"{filename} downloaded with following info: \n{headers}")
            else:
                logger.info(f"File already exists of size :{get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
            """
            zip_file_path: str
            extract the zip file into the data directory
            function returns None
            """
            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path, exist_ok=True)   
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)

In [115]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-08-17 16:06:28,565: INFO: Yaml file: config\config.yaml loaded successfully.]
[2025-08-17 16:06:28,573: INFO: Yaml file: params.yaml loaded successfully.]
[2025-08-17 16:06:28,577: INFO: created directory at: artifacts]
[2025-08-17 16:06:28,583: INFO: created directory at: artifacts/data_ingestion]


[2025-08-17 16:06:30,627: INFO: artifacts\data_ingestion\data.zip downloaded with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: C1C1:CF4CB:15BA8B2:1980038:68A1E1E6
Accept-Ranges: bytes
Date: Sun, 17 Aug 2025 14:06:30 GMT
Via: 1.1 varnish
X-Served-By: cache-mxp6921-MXP
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1755439590.236069,VS0,VE377
Vary: Authorization,Accept-Encoding
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: f5585cf9e2378cd7c72378b087a0523b5e27d562
Expires: Sun, 17 Aug 2025 14:11:30 GMT
Source-Age: 0

]
